<a href="https://colab.research.google.com/github/Prahlad108/ME592-HW-Projects/blob/main/HW2_Engineering_Design.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Homework 2 - Engineering Design Team**

Note that the code is not finished as we were having major issues, we will hopefully have an improved version for 3/10/2022 as per the teacher mentioning that it could be handed in then if we needed more time, however we will sumbit what we have to canvas first just incase

In [ ]:
#Moved from the above drive importing code that so that it could be run without interference
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import numpy as np
import pandas as pd
from sklearn import datasets


In [ ]:
#Alex K
#@DATE: 3/6/2022
#@PURPOSE: This installs geomdl, the code that is commented out was supposed to import NURBS but I think it is not needed
!pip3 install geomdl
  # Importing NURBS module
#from geomdl.core import NURBS
  # Importing visualization module
#from geomdl.visualization import VisMPL as vis

I'm not sure why but the above drive mounting had issues for me, what I did below worked for me, this part isn't terribly important as long as we can all get it working to run
-Dan

In [ ]:
#Dan
#@DATE: 3/8/2022
#@PURPOSE: This block will mount your google colab drive. Successful execution
#         of the remainder of this code depends on a file structure of...
#         /MyDrive/Colab Notebooks/ME 592/HW2/HW2.ipynb
#         /MyDrive/Colab Notebooks/ME 592/HW2/data/[input_geometry/output_geometry]


import os, sys
from google.colab import drive
# This mouunts your google drive to the current runtime
drive.mount('/content/mnt', force_remount=True)
# We define a notebook path
nb_path = '/content/notebooks'
# We create a symbolic link from our drive's default "Colab Notebooks" folder to nb_path
os.symlink('/content/mnt/My Drive/Colab Notebooks', nb_path)
# Insert nb path
sys.path.insert(0, nb_path)

Mounted at /content/mnt


Below I just set up relative paths to where all of our data is stored. I made the assumption that we could all copy-paste the entire "data" folder into the same directory level as our HW2. I copied the folder structure Brian was using above and simply added it into my google drive since the relative paths will associate to our own google drive, not Brian's who own's the colab session. -Dan

In [ ]:
#Dan
#@DATE: 3/8/2022
#@PURPOSE: Establish filepaths for digging into directories after symbolic link

# Filepath to the data folder
dataFolder = nb_path + '/ME 592/HW2/data'
#dataFolder = os.path.join(nb_path,'/ME 592/HW2/data')
# Filepath to the input data folder
inputFolder = dataFolder + '/input_geometry'
#inputFolder = os.path.join(dataFolder, 'input_geometry')
# Filepath to the output data folder
outputFolder = dataFolder + '/final_geometry'
#outputFolder = os.path.join(dataFolder,'final_geometry')

This is the meat of part 1 of the homework. I stored everything into one large data list of tuples. The tuples are formatted the same way that he recommended in the HW assignment. (input,output) where input is (mesh,time,temp,pres) and output is (mesh)
each of the meshes are currently 3x12x17 (reversed from what he stated). We can mirrior this back to 17x12x3 pretty easily though

Notes: I made an assumption that the output is to be read as the first 17 data lines are elements A[0-16][0][0], then the following 17 data lines would be A[17-31][1][0]... and so on until you loop around to the the next input mesh. 

If anyone has addt'l information on the anticipation of how to read this, I can help adjust, but I couldn't find anything that specified how to read it and made an assumption to get us started.

In [ ]:
#@AUTHOR: Dan
#@DATE: 3/8/2022
#@PURPOSE: 
import natsort

# Compile a list of the run directories in the input geometry folder
runDirList = [dir for dir in os.listdir(inputFolder)]

# Sorting the rundir list
runDirList = natsort.natsorted(runDirList)

# Compile a list of the final geometry data files
outputList = [os.path.join(outputFolder,f) for f in os.listdir(outputFolder) if os.path.isfile(os.path.join(outputFolder,f))]

# List of Pressures [mmHg]
#pressList = ['76mmHg','80mmHg','84mmHg']
pressList = ['p1','p2','p3']
# List of Temperatures [Kelvin]
#tempList = ['300K','350K','400K','450K','500K']
tempList = ['t1','t2','t3','t4','t5']
# List of Times [sec]
timeList = ['80','140']
# Initializing 3-D Data Array structure
dataArray = [[[[ [] for _ in range(len(runDirList))] for _ in range(len(tempList))] \
              for _ in range(len(pressList))] for _ in range(len(timeList))]

dataArrayParams = [[[[ [] for _ in range(len(runDirList))] for _ in range(len(tempList))] \
              for _ in range(len(pressList))] for _ in range(len(timeList))]

# Data Array Structure Key 
dataArrayKey = [[[[ [time,press,temp,run] for run in runDirList] for temp in tempList] for press in pressList] for time in timeList]

# OBE dataList
dataList = []

# loop through each run directory
for runDir in runDirList:
  
  # Run Index from input file naming convention
  runIdx = runDir[3:]
  # Creating OS file path of the run directory
  runDirPath = os.path.join(inputFolder, runDir)

  # Checking for each file in the run directory
  inputFileList = [os.path.join(runDirPath,f) for f in os.listdir(runDirPath) if os.path.isfile(os.path.join(runDirPath,f))]

  # Initializing input geometry 17x12x3 array
  inputGeometry = [[[ [] for _ in range(3)] for _ in range(12)] for _ in range (17)]
  
  # Input Mesh Index Counters
  i = 0 
  j = 0 
  k = 0

  for input in inputFileList:
    
    with open(input) as file:
      lines = [line.rstrip('\n') for line in file]

    for l in range(len(lines)):

      if l > 4 and l != len(lines)-1:
        
        k = (l-5) % 17

        if k == 0 and l > 5:
          j = j + 1

        line = lines[l].split()[:-1]

        line = [float(n) for n in line]

        inputGeometry[k][j][i] = line

    i = i + 1
    j = 0
    k = 0

  for time in timeList:
    for press in pressList:
      for temp in tempList:
        # Mapping the input to output geometry files
        outputFiles = [f for f in outputList if f.split('_')[-1] == str(runIdx) \
                       and f.split('_')[-2] == press \
                       and f.split('_')[-3] == temp \
                       and f.split('_')[-4] == time]

        outputGeometry = [[[ [] for _ in range(3)] for _ in range(12)] for _ in range (17)]

        # Output Mesh Index Counters
        i = 0 
        j = 0 
        k = 0

        if len(outputFiles) != 1:
          #print('Output Mapping contains multiple solutions')
          #print('missing data [run#, time, press, temp]:',runIdx,time,press,temp)
          outputGeometry = [[[ [-9999] for _ in range(3)] for _ in range(12)] for _ in range (17)]

        else:
          with open(outputFiles[0]) as file:
            lines = [line.rstrip('\n') for line in file]
          
          for l in range(len(lines)):
            
            # Skipping the first line of meta data
            if l == 0:
              continue
            
            line = lines[l].split()

            k = (l-1) % 17

            if k == 0 and l > 5:
              j = j + 1

            if (l-1)%(12*17) == 0 and l > 2:
              i = i + 1
              j = 0
              k = 0

            line = [float(n) for n in line]

            outputGeometry[k][j][i] = line

          # Format data as a tuple of input and outputs and lists
          input = [inputGeometry, temp, press]
          output = [outputGeometry]
          dataElement = (input,output)
        
        # Data array without additional Tuple Parameters
        dataArrayParams[timeList.index(time)][pressList.index(press)][tempList.index(temp)][runDirList.index(runDir)] = dataElement
        
        input = [inputGeometry]
        dataElement = (input,output)
        # Data array with pressure, temperature in input tuple parameters
        dataArray[timeList.index(time)][pressList.index(press)][tempList.index(temp)][runDirList.index(runDir)] = dataElement
        

In [ ]:
# Example structure of compiled 3-D data array of (input,output) tuple run-keys
import os
# List of Pressures [mmHg]
#pressList = ['76mmHg','80mmHg','84mmHg']
pressList = ['p1','p2','p3']

# List of Temperatures [Kelvin]
#tempList = ['300K','350K','400K','450K','500K']
tempList = ['t1','t2','t3','t4','t5']

# List of Times [sec]
timeList = ['80','140']

import natsort
runDirList = [dir for dir in os.listdir(inputFolder)]
runDirList = natsort.natsorted(runDirList)

dataArrayKey = [[[[ [time,press,temp,run] for run in runDirList] for _ in tempList] for _ in pressList] for _ in timeList]

dataArrayExample = [[[[ [] for _ in range(len(runDirList))] for _ in range(len(tempList))] \
              for _ in range(len(pressList))] for _ in range(len(timeList))]

# Example of keying into the data array:
# Time 80, Temperature 1, Pressure 2 runs
# dataArrayExample[timeList.index('80')][pressList.index('p1')][tempList.index('t1')][:]

'''
To parse through the files I kept the list correlated to the run output file 
naming format, see above for what each 'p1', or 't1' if desired we can also 
easily do a replace across the data structure to what those "truely" mean 

Last we can do list comprehension on this such as... below to get all times for 
p1, t1, and run1

NOTE: the rundir list are named as folder structures are, e.g. "run1"... "run64"
'''

# dataArrayExample[:][pressList.index('p1')][tempList.index('t1')][runDirList.index('run1')]

'''
A given element in the 4-D array is a Tuple of (input,output)

To key into the tuple we then need to key even deeper into the tuple

I'm going to explain this relative to the tuple pulled out of the array, but
this can easily be done by thinking of our 4-D array as actually a 4-D array of
2-D arrays of 3-D arrays.

remember here: the input = [ [17x12x3], [1,], [1,] ]
          and the output = [ [17x12x3] ]
          identical to how HW2 part 1 is showing if that explanation helps more
'''

# Pulling an (input,output) tuple out of our 4-D run-matrix
myTuple = dataArray[0][0][0][0]

# Looking at the input portion of the tuple
myTupleInput = myTuple[0]

# Looking at the input press, temp of the tuple
myTupleInputGeometry = myTuple[0][1]
myTupleInputGeometry = myTuple[0][2]

# Looking at the input geometry of the tuple
myTupleInputGeometry = myTuple[0][0]

# Looking at the output geometry of the tuple
myTupleOutputGeometry = myTuple[1][0]


IndexError: ignored

Below we are trying to take the data Dan created and perform the PCA whitening on it for problem 2.

In [ ]:
#Alex K and Prahlad P
#@DATE: 3/9/2022
#@PURPOSE: Attempting to solve problem 2
from sklearn.decomposition import PCA
import numpy as np

#Used to adjust n_components parameter for pca whitening (it also adjust some array sizes as needed for this as well)
#Causes pca whitening to put out a 17xn data set
n = 2

#Create empty 8D array where we have the 17xn data at the end which will contain the PCA whitened data for xyz and specific smesh files
#The number for the sixth component of dataArrayWhitened corresponds to the number used for n_components, will be called PCA whitened data 2 in structure
#dataArrayWhitened Structure [time, pressure, temperature, run, PCA whitened data 1, PCA whitened data 2,xyz channels, which smesh file]
dataArrayWhitened = np.zeros([2,3,5,64,17,n,3,3])

#Data array values size is 2x3x5x64
a = 0 
b = 0
c = 0
d = 0
for a in range(2):
  for b in range(3):
    for c in range(5):
      for d in range(64):
###############################################################################
        #Create array from list of size 17x12x3x3 for doing transform on
        #arr structure = [image data 1, image data 2, xyz channels, smesh file]
        arr = np.array(dataArray[a][b][c][d][0][0])

        #Create array pcaOut used for storing pca results which get inerted into the output dataArrayWhitened 8D array with the PCA whitened Data
        #Note that the second number of pcaOut corresponds to the number set for n_components
        pcaOut = np.zeros([17,n,3,3])
        
        #Loops through the three smesh files
        i = 0
        for i in range(3):
          #Does the PCA transform on the x, y, and z planes of a given smesh file i
          pca = PCA(n_components=n,svd_solver='auto', whiten=True)
          pcaOut[:,:,0,i] = pca.fit_transform(arr[:,:,0,i])
          pcaOut[:,:,1,i] = pca.fit_transform(arr[:,:,1,i])
          pcaOut[:,:,2,i] = pca.fit_transform(arr[:,:,2,i])
          i = i + 1
        dataArrayWhitened[a][b][c][d] = pcaOut
###############################################################################
        d = d + 1
      d = 0
      c = c + 1
    c = 0
    b = b + 1
  b = 0
  a=a+1

print(dataArrayWhitened.shape)

(2, 3, 5, 64, 17, 2, 3, 3)


In [ ]:
#Alex K and Prahlad P
#@DATE: 3/9/2022
#@PURPOSE: Attempting to solve/work on Problem 3
!pip install -q keras 

from google.colab import drive
import keras
import pandas as pd
import numpy as np
from scipy import linalg
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.metrics import pairwise_distances
from sklearn.datasets import load_digits
from scipy.spatial.distance import squareform

import seaborn as sns

drive.mount('/content/drive/')
outputPath = '/content/drive/MyDrive/ME592/HW2/data/final_geometry'
#!ls '/content/drive/MyDrive/ME592/HW2/data/final_geometry'

sns.set(rc={'figure.figsize':(11.7,8.27)})
palette = sns.color_palette('bright', 10)

X,y = load_digits(return_X_y=True)
MACHINE_EPSILON = np.finfo(np.double).eps
n_components = 2
perplexity = 30

def fit(X):
  n_samples = X.shape[0]
  dist = pairwise_distances(X, metric='euclidean', squared=True) #euclidian distance
  P = joint_probabilities(distances=dist, desired_perplexity=perplexity, verbose=False) #joint probabilities from distances p_i,j
  X_embedded = 1e-4*(np.random.mtrand.rand.randn(n_samples, n_components).astype(np.float32))
  dof = n_components-1
  return _tsne(P, dof, n_samples, X_embedded=X_embedded)

def _tsne(P, dof, n_samples, X_embedded):
  params = X_embedded.ravel()
  obj_func =_kl_divergence
  params = _gradient_descent(obj_func, params, [P, dof, n_samples, n_components])
  return X_embedded

g = np.random.rand(17,12)
np.ravel(g)

def _kl_divergence(params, P, dof, n_samples, n_components):
  X_embedded = params.reshape(n_samples, n_components)
  d = pdist(X_embedded, 'sqeuclidean')
  d /= dof
  d += 1
  d **= (d + 1.0)/-2.0
  q = np.maximum(d/2.0*np.sum(d), MACHINE_EPSILON)
  kl_divergence = 2.0 * np.dot(P, np.log(np.maximum(P, MACHINE_EPSILON) / q))

  grad = np.ndarray((n_samples, n_components), dtype=params.dtype)
  pqd = squareform((P-q)*d)
  for i in range(n_samples):
    grad[i] = np.dot(np.ravel(pqd[i], order='K'), X_embedded[i]-X_embedded)
  grad = grad.ravel()
  v = 2.0*(dof+1.0)/dof
  grad *= v

  return kl_divergence, grad

def _gradient_descent(obj_func, h0, args, it=0, n_iter=1000, n_iter_check=1, n_iter_without_progress=300, momentum=0.75, learning_rate=200.0, min_gain=0.01, min_grad_norm=1e-7):
    h = h0.copy().ravel()
    update = np.zeros_like(h)
    gains = np.ones_like(h)
    err = np.finfo(np.float).max
    best_err = np.finfo(np.float).max
    best_iter = i = it

    for i in range(it, n_iter):
      err, grad=obj_func(h, *args)
      grad_norm = linalg.norm(grad)

      increase = update*grad<0.0
      decrease = np.invert(increase)
      gains[increase] += 0.2
      gains[decrease] *= 0.8
      np.clip(gains, min_gain, np.inf, out=gains)
      grad *= gains
      update = momentum*update - learning_rate*grad
      h += update

      print('[t-sne] Iteration %d: err=%0.7f', 'gradient norm=%0.7f' % (i+1, err, grad_norm))

      if err < best_err:
        best_err = err
        best_iter = 1
      elif i - best_iter > n_iter_without_progress:
        break
      if grad_norm <= min_grad_norm:
        break
      return h

X_embedded = fit(X)
sns.scatterplot(X_embedded[:,0], X_embedded[:,1], hue=y, legend='full', palette=palette)






